# 01 · LangGraph 고급 패턴 (Advanced Integration)

## 🎯 이 노트북의 목적

**강사 시연용 노트북**입니다. 프리 세션에서 고급 패턴을 시연하며, 학생들은 이후 main-practice에서 직접 구현합니다.

### 📚 00번에서 배운 것 (복습)
- **State**: 노드들이 공유하는 데이터 저장소
- **Node**: State를 처리하는 함수 (필요한 필드만 업데이트)
- **Graph**: 노드들을 연결한 워크플로우 (add_node, add_edge, compile, invoke)

### 🚀 01번에서 배울 것 (고급 패턴 4가지)

#### 1️⃣ **Annotated State** (리스트 누적)
```python
audit_log: Annotated[List[str], operator.add]
```
- **문제**: 여러 노드가 리스트에 추가하면 덮어쓰기됨
- **해결**: `operator.add`로 자동 누적

#### 2️⃣ **Command** (동적 라우팅)
```python
return Command(goto='next_node', update={...})
```
- **문제**: 조건에 따라 다음 노드를 동적으로 선택
- **해결**: `Command`로 실행 흐름 제어

#### 3️⃣ **Fan-in** (병렬 처리 후 합치기)
```python
technical → \
policy →    } → fan_in → 최종
service →  /
```
- **문제**: 여러 노드의 결과를 하나로 합치기
- **해결**: 여러 엣지가 하나의 노드로 모이는 패턴

#### 4️⃣ **interrupt** (Human-in-the-Loop)
```python
feedback = interrupt({...})
result = app.invoke(Command(resume=feedback))
```
- **문제**: 중요한 결정은 사람이 검토 필요
- **해결**: `interrupt()`로 워크플로우 일시정지, `resume`으로 재개

### 🎬 시나리오: 고객 문의 자동 처리 시스템
- 고객 문의 → 자동 분류 (기술/정책/서비스)
- 3개 팀이 동시에 보고서 작성 (Fan-in)
- 정책 이슈는 인간 검토 필요 (interrupt)

### ⏱️ 예상 시연 시간: 15분

## Step 0. 환경 준비
1. 의존성 설치가 필요하면 아래 `pip` 셀을 실행하세요. (이미 설치되어 있다면 건너뜁니다.)
2. 이어지는 import 셀을 반드시 실행해야 이후 코드에서 타입과 유틸 함수를 인식할 수 있습니다.

In [ ]:
%pip install -q langgraph langchain langchain-teddynote

In [ ]:
from typing import TypedDict, Dict, List, Annotated

from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver
from langchain_teddynote.graphs import visualize_graph
import operator

print("✅ 필요한 라이브러리 import 완료!")

## Step 1. Annotated State (리스트 누적)

### 🎯 고급 패턴 #1: operator.add로 리스트 자동 누적

#### 문제 상황:
00번에서 배운 State는 **덮어쓰기** 방식입니다:
```python
# 기본 State
class SimpleState(TypedDict):
    logs: List[str]  # 기본: 덮어쓰기

# Node 1
def node1(state):
    return {"logs": ["Step 1"]}  # logs = ["Step 1"]

# Node 2
def node2(state):
    return {"logs": ["Step 2"]}  # logs = ["Step 2"]로 덮어씀!
```

❌ **문제**: Node 2가 Node 1의 로그를 덮어써버림!

#### 해결책: Annotated + operator.add
```python
from typing import Annotated
import operator

class AdvancedState(TypedDict):
    logs: Annotated[List[str], operator.add]  # 누적 모드!

# Node 1
def node1(state):
    return {"logs": ["Step 1"]}  # logs = ["Step 1"]

# Node 2
def node2(state):
    return {"logs": ["Step 2"]}  # logs = ["Step 1", "Step 2"]로 누적!
```

✅ **해결**: `operator.add`가 자동으로 리스트를 합쳐줌!

### 💡 실무 활용:
- **audit_log**: 워크플로우 실행 로그 추적
- **messages**: 대화 내역 누적
- **results**: 여러 노드의 처리 결과 수집

---

아래 셀에서는 `audit_log`를 `Annotated[List[str], operator.add]`로 정의해서,  
각 노드가 로그를 남기면 자동으로 누적되는 것을 확인합니다.

In [ ]:
class IntegratedState(TypedDict):
    # 고객 문의 정보
    customer_query: str
    category: str  # technical / policy / service
    
    # 각 팀의 보고서
    technical_report: str
    policy_report: str
    service_report: str
    
    # 응답 생성
    draft_response: str
    final_response: str
    
    # 인간 검토
    review_feedback: str
    approval_status: str  # pending / auto / approved
    
    # ⭐ 핵심: Annotated로 리스트 누적!
    audit_log: Annotated[List[str], operator.add]


def initial_state(query: str) -> IntegratedState:
    """초기 State 생성 헬퍼 함수"""
    return IntegratedState(
        customer_query=query,
        category='',
        technical_report='',
        policy_report='',
        service_report='',
        draft_response='',
        final_response='',
        review_feedback='',
        approval_status='pending',
        audit_log=[],  # 빈 리스트로 시작
    )

print("✅ IntegratedState 정의 완료!")
print("   - 고객 문의 라우팅 시스템")
print("   - audit_log는 Annotated로 자동 누적")

---

### 🎯 실습 1: Annotated State 직접 만들어보기

배송 추적 시스템을 만들고 있습니다. 여러 노드가 로그를 추가해야 하는데, 로그가 덮어씌워지지 않도록 State를 정의하세요!

**요구사항:**
- `package_id: str` (택배 번호)
- `status: str` (현재 상태)
- `history: Annotated[List[str], operator.add]` ← 로그 누적!

아래 셀에서 직접 작성해보세요.

In [ ]:
# 여기를 수정하세요! 👇
# TODO: TrackingState를 정의하세요

from typing import TypedDict, List, Annotated
import operator

class TrackingState(TypedDict):
    package_id: str
    status: str
    # 여기에 history 필드를 추가하세요! (Annotated 사용)
    pass  # 이 줄을 지우고 작성하세요

# 테스트
test_state = TrackingState(
    package_id="PKG-001",
    status="배송중",
    history=[]
)

print("✅ State 정의 완료!")
print(f"   package_id: {test_state['package_id']}")
print(f"   history 타입: {TrackingState.__annotations__.get('history', 'X')}")

# 정답 확인
if 'history' in TrackingState.__annotations__:
    anno = str(TrackingState.__annotations__['history'])
    if 'Annotated' in anno and 'operator.add' in anno:
        print("\n🎉 정답! Annotated[List[str], operator.add]로 누적됩니다!")
    else:
        print("\n❌ 힌트: Annotated[List[str], operator.add]를 사용하세요")
else:
    print("\n💡 힌트: history 필드를 추가하세요")

<details>
<summary>📝 정답 보기 (클릭)</summary>

```python
from typing import TypedDict, List, Annotated
import operator

class TrackingState(TypedDict):
    package_id: str
    status: str
    history: Annotated[List[str], operator.add]  # ← 핵심!
```

**왜 필요한가?**
```python
# ❌ 기본 State (Annotated 없음)
history: List[str]
# Node 1: return {"history": ["접수"]}
# Node 2: return {"history": ["출고"]}
# 결과: ["출고"]  ← 덮어씀!

# ✅ Annotated State
history: Annotated[List[str], operator.add]
# Node 1: return {"history": ["접수"]}
# Node 2: return {"history": ["출고"]}
# 결과: ["접수", "출고"]  ← 누적!
```

</details>

In [ ]:
example_state = initial_state('충전기가 고장나서 도움이 필요해요')
print("📋 초기 상태:")
for key, value in example_state.items():
    print(f"   {key}: {value}")

---

### 🎯 퀴즈 1: Annotated State가 필요한 경우는?

여러 노드가 로그를 추가할 때, 어떤 State 정의가 맞을까요?

**A)** `logs: List[str]` (기본)  
**B)** `logs: Annotated[List[str], operator.add]`  
**C)** `logs: str`



In [ ]:
# 여기를 수정하세요! 👇
# TODO: 정답을 선택하세요

my_answer = ""  # "A", "B", "C"

if my_answer == "B":
    print("✅ 정답! operator.add로 리스트가 자동 누적됩니다!")
elif my_answer:
    print("❌ 다시 생각해보세요. 로그가 덮어씌워지면 안 돼요!")
else:
    print("💡 힌트: 여러 노드의 로그를 모두 보존하려면?")

<details>
<summary>📝 정답 보기 (클릭)</summary>

**정답: B** `logs: Annotated[List[str], operator.add]`

**해설:**
- A) ❌ 기본 State → 덮어쓰기 (마지막 로그만 남음)
- **B) ✅ Annotated + operator.add → 자동 누적**
- C) ❌ 문자열 (리스트가 아님)

**핵심:**
```python
# Node 1
return {"logs": ["Step 1"]}

# Node 2
return {"logs": ["Step 2"]}

# 결과:
# A) logs = ["Step 2"]  ← 덮어씀 ❌
# B) logs = ["Step 1", "Step 2"]  ← 누적 ✅
```

</details>

## Step 2. Command (동적 라우팅)

### 🎯 고급 패턴 #2: Command로 실행 흐름 제어

#### 00번에서 배운 방식의 한계

```python
# 00번: set_conditional_entry_point
def decide_path(state):
    if len(state["input"]) <= 5:
        return "simple"  # ← 노드 이름만 리턴
    return "complex"

workflow.set_conditional_entry_point(decide_path, {...})
```

**문제**: State 업데이트가 안 됨!  
분류 결과를 다음 노드에서 재사용하려면?

---

### ✅ 해결책: Command

```python
from langgraph.types import Command

def router_node(state):
    category = classify(state['query'])  # 분류 1번만!
    
    return Command(
        goto="tech_team",        # ← 다음 노드 지정
        update={                 # ← State 업데이트!
            'category': category,
            'audit_log': [f"라우터: {category}"]
        }
    )

# 다음 노드에서 바로 사용
def tech_team(state):
    category = state['category']  # ← 저장된 값 사용!
```

**장점:**
- `goto`: 다음 노드 동적 지정
- `update`: State에 결과 저장 → 재사용
- `audit_log`: 실행 경로 추적 (Annotated와 연계!)

---

아래 셀에서 실제로 Command를 써봅니다.

In [ ]:
def router_node(state: IntegratedState) -> Command:
    """
    고객 문의를 분류해서 적절한 팀으로 라우팅
    
    ⭐ Command 사용:
    - goto: 다음 실행할 노드 동적 지정
    - update: State에 category와 audit_log 추가
    """
    query = state['customer_query']
    
    # 간단한 키워드 분류
    if '고장' in query or '오류' in query:
        category = 'technical'
        target = 'technical_team'
    elif '환불' in query or '취소' in query:
        category = 'policy'
        target = 'policy_team'
    else:
        category = 'service'
        target = 'service_team'
    
    # ⭐ Command로 노드 선택 + State 업데이트
    return Command(
        goto=target,
        update={
            'category': category,
            'audit_log': [f'라우터: {category} 선택'],
        }
    )


# 데모용 간단한 라우터 그래프
print("🧪 라우터 데모:")
print("   키워드로 문의 유형을 자동 분류합니다")
print()

router_graph = StateGraph(IntegratedState)
router_graph.add_node('router', router_node)
router_graph.add_edge(START, 'router')
router_graph.add_edge('router', END)
router_app = router_graph.compile()

visualize_graph(router_app)

# 테스트
for query in ['충전기가 고장났어요', '환불 처리 부탁드려요', '상담원 연결 부탁드려요']:
    result = router_app.invoke(initial_state(query))
    print(f"문의: {query}")
    print(f"   → 분류: {result['category']}")
    print(f"   → 로그: {result['audit_log']}")
    print()

print("💡 관찰: audit_log가 자동으로 누적 (Annotated 덕분!)")

---

### 🎯 실습 2: Command로 라우터 만들기

고객 문의를 분류해서 적절한 팀으로 보내는 router를 작성하세요!

**요구사항:**
- "배송" 키워드 → delivery_team
- "결제" 키워드 → payment_team
- 나머지 → support_team
- category를 State에 저장 (update 사용!)

아래 셀에서 직접 작성해보세요.

In [ ]:
# 여기를 수정하세요! 👇
# TODO: simple_router 함수를 완성하세요

from langgraph.types import Command

class SimpleState(TypedDict):
    query: str
    category: str

def simple_router(state: SimpleState) -> Command:
    '''고객 문의를 분류하는 라우터'''
    query = state['query']
    
    # 여기에 분류 로직을 작성하세요! 👇
    if '배송' in query:
        category = 'delivery'
        target = 'delivery_team'
    # elif ... (결제, 나머지 추가)
    
    # Command를 리턴하세요!
    # goto=..., update={...}
    pass  # 이 줄을 지우고 Command를 리턴하세요

# 테스트
try:
    result = simple_router({"query": "배송 조회 부탁드려요", "category": ""})
    
    if isinstance(result, Command):
        print("✅ Command 리턴 성공!")
        if hasattr(result, 'update') and 'category' in result.update:
            print(f"   분류: {result.update['category']}")
            print(f"   다음 노드: {result.goto}")
            print("\n🎉 정답!")
        else:
            print("\n❌ update에 category를 추가하세요!")
    else:
        print("\n❌ Command를 리턴하세요!")
except:
    print("💡 힌트: return Command(goto=..., update={...})")

<details>
<summary>📝 정답 보기 (클릭)</summary>

```python
from langgraph.types import Command

def simple_router(state: SimpleState) -> Command:
    query = state['query']
    
    if '배송' in query:
        category = 'delivery'
        target = 'delivery_team'
    elif '결제' in query:
        category = 'payment'
        target = 'payment_team'
    else:
        category = 'support'
        target = 'support_team'
    
    return Command(
        goto=target,
        update={'category': category}
    )
```

**핵심:**
- `goto`: 다음 노드 동적 선택
- `update`: State에 결과 저장 → 다음 노드에서 재사용!

</details>

### 💡 참고: Conditional Edge와 함께 사용하기

#### ❓ "conditional edge도 있던데, Command랑 뭐가 다름?"

**핵심**: 둘 다 쓸 수 있고, Command가 우선권을 가짐!

```
[ 노드 실행 ]
    ↓
Command 반환? 
  YES → Command 우선 적용 ✅
  NO  → Conditional Edge 작동 ✅
```

#### 실무 패턴: "기본은 Edge, 예외는 Command"

```python
# 기본 플로우: Edge로 선언 (시각적으로 명확)
graph.add_conditional_edges(
    "review",
    lambda s: "approved" if s["score"] > 0.8 else "pending",
    {"approved": "send", "pending": "wait"}
)

# 노드 내부: 예외 케이스만 Command
def review_node(state):
    if "urgent" in state["query"]:
        # 긴급! Edge 무시하고 즉시 우회
        return Command(goto="priority_team", update={...})
    
    # 일반 케이스: Command 없음 → Edge 작동
    return state
```

**언제 뭘 쓸까?**
- **간단한 분기**: conditional edge
- **State 업데이트 필요**: Command
- **audit_log 추적**: Command
- **둘 다 필요**: 병행! (기본 Edge + 예외 Command)

---

이 노트북에서는 Command에 집중하지만,  
실무에서는 **둘 다 섞어** 씁니다! 🔥

## Step 3. Fan-in (자동 State 병합)

### 🎯 고급 패턴 #3: 여러 노드 → 자동 병합

#### 시나리오:
3개 팀이 동시에 작업하면, 결과를 어떻게 합칠까?

```
START
  ├→ technical_team (보고서 작성)
  ├→ policy_team (보고서 작성)
  └→ service_team (보고서 작성)
       ↓
    ??? (어떻게 합치지?)
```

#### 💡 LangGraph의 자동 병합

**놀라운 사실**: fan_in 노드가 필요 없습니다!

```python
# 3개 팀 → 하나의 노드로 엣지 연결
graph.add_edge('technical_team', 'next_node')
graph.add_edge('policy_team', 'next_node')
graph.add_edge('service_team', 'next_node')

# → LangGraph가 자동으로 State 병합! ✨
```

#### 병합 규칙:
- **일반 필드**: 마지막 업데이트 값 사용
- **Annotated 필드**: reducer로 병합 (operator.add 등)

#### 예시:
```python
# technical_team: {'technical_report': '...', 'audit_log': ['기술팀']}
# policy_team: {'policy_report': '...', 'audit_log': ['정책팀']}
# service_team: {'service_report': '...', 'audit_log': ['서비스팀']}

# 자동 병합 후:
# {
#   'technical_report': '...',
#   'policy_report': '...',
#   'service_report': '...',
#   'audit_log': ['기술팀', '정책팀', '서비스팀']  ← Annotated로 누적!
# }
```

### 🎯 실무 활용:
- **병렬 API 호출**: 여러 외부 API 동시 호출
- **멀티 모델**: 여러 LLM 응답 종합
- **분산 처리**: 큰 작업을 나눠서 처리

---

아래 셀에서는 3개 팀이 동시에 작업하고, LangGraph가 자동으로 병합하는 것을 봅니다.

In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 3개 전문 팀 노드 정의
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
def technical_team_node(state: IntegratedState) -> Dict:
    """기술팀: 기술적 해결 방안"""
    report = f"""[기술팀 보고서]
문제: {state['customer_query']}
조치: 전원 재연결, 펌웨어 업데이트, 하드웨어 점검"""
    
    return {
        'technical_report': report,
        'audit_log': ['기술팀 보고서 작성'],
    }


def policy_team_node(state: IntegratedState) -> Dict:
    """정책팀: 환불/교환 정책"""
    report = f"""[정책팀 보고서]
문의: {state['customer_query']}
조치: 환불/교환 기준 검토, 증빙 서류 요청"""
    
    return {
        'policy_report': report,
        'audit_log': ['정책팀 보고서 작성'],
    }


def service_team_node(state: IntegratedState) -> Dict:
    """서비스팀: 고객 응대 메시지"""
    report = """[서비스팀 보고서]
공감 표현: 불편을 드려 죄송합니다
후속 지원: 24시간 상담 채널 안내"""
    
    return {
        'service_report': report,
        'audit_log': ['서비스팀 보고서 작성'],
    }


# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Fan-in 데모: 자동 병합 확인
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print("🧪 Fan-in 자동 병합 데모:")
print("   3개 팀 → 자동 병합 (fan_in_node 없음!)")
print()

# 간단한 종료 노드 (병합 결과만 보여주기)
def show_merged_node(state: IntegratedState) -> Dict:
    """병합된 State 확인"""
    print("✅ 자동 병합 완료!")
    print(f"   technical_report: {'있음' if state['technical_report'] else '없음'}")
    print(f"   policy_report: {'있음' if state['policy_report'] else '없음'}")
    print(f"   service_report: {'있음' if state['service_report'] else '없음'}")
    print(f"   audit_log: {state['audit_log']}")
    return {}

fan_in_graph = StateGraph(IntegratedState)
fan_in_graph.add_node('technical_team', technical_team_node)
fan_in_graph.add_node('policy_team', policy_team_node)
fan_in_graph.add_node('service_team', service_team_node)
fan_in_graph.add_node('show_merged', show_merged_node)

# START → 3개 팀 (Fan-out)
fan_in_graph.add_edge(START, 'technical_team')
fan_in_graph.add_edge(START, 'policy_team')
fan_in_graph.add_edge(START, 'service_team')

# 3개 팀 → show_merged (자동 Fan-in!)
fan_in_graph.add_edge('technical_team', 'show_merged')
fan_in_graph.add_edge('policy_team', 'show_merged')
fan_in_graph.add_edge('service_team', 'show_merged')

fan_in_graph.add_edge('show_merged', END)

fan_in_app = fan_in_graph.compile()

print("📊 Fan-in 그래프:")
visualize_graph(fan_in_app)

result = fan_in_app.invoke(initial_state('충전기가 고장나서 도움이 필요해요'))

print("\n💡 핵심:")
print("   - fan_in_node 없이도 자동 병합!")
print("   - 각 팀의 필드가 하나로 합쳐짐")
print("   - audit_log는 Annotated로 3개 모두 누적")

---

### 🎯 퀴즈: Fan-in 병합 규칙

3개 노드가 동시에 실행되고 하나로 합쳐질 때, 결과는?

**상황:**
```python
# Node A: return {"status": "A완료", "logs": ["A"]}
# Node B: return {"status": "B완료", "logs": ["B"]}
# Node C: return {"status": "C완료", "logs": ["C"]}

# State 정의:
class State(TypedDict):
    status: str  # 일반 필드
    logs: Annotated[List[str], operator.add]  # Annotated
```

**병합 후 status는?**

**A)** "A완료"  
**B)** "B완료" 또는 "C완료" (마지막 업데이트)  
**C)** "A완료, B완료, C완료" (합쳐짐)

In [ ]:
# 여기를 수정하세요! 👇
# TODO: 정답을 선택하세요

my_answer = ""  # "A", "B", "C"

if my_answer == "B":
    print("✅ 정답!")
    print("   일반 필드는 마지막 업데이트 값만 남습니다!")
    print("   Annotated 필드(logs)만 누적됩니다!")
elif my_answer:
    print("❌ 다시 생각해보세요!")
else:
    print("💡 힌트: 일반 필드 vs Annotated 필드의 병합 규칙은?")

<details>
<summary>📝 정답 보기 (클릭)</summary>

**정답: B** - "B완료" 또는 "C완료" (마지막 업데이트)

**병합 규칙:**

```python
# 일반 필드: 마지막 값만 남음 (덮어쓰기)
status: str
# A, B, C 중 마지막으로 실행된 것만 남음!

# Annotated 필드: reducer로 병합 (누적)
logs: Annotated[List[str], operator.add]
# ["A", "B", "C"] 모두 누적!
```

**결과:**
```python
{
    "status": "B완료"  # 또는 "C완료" (실행 순서에 따라)
    "logs": ["A", "B", "C"]  # 모두 누적!
}
```

**핵심:** 일반 필드는 덮어쓰기, Annotated만 누적!

</details>

## Step 4. interrupt (Human-in-the-Loop)

### 🎯 고급 패턴 #4: 워크플로우 일시정지 & 재개

#### 시나리오:
민감한 결정(환불, 정책 변경 등)은 사람이 검토해야 합니다.

```
START → draft → review_gate
                     ↓
                 interrupt! ⏸️
                     ↓
    (사람이 검토 후 피드백 입력)
                     ↓
            Command(resume=...) ▶️
                     ↓
              finalize → END
```

#### 💡 interrupt의 특징

```python
from langgraph.types import interrupt, Command

def review_gate(state):
    if needs_human_review(state):
        # ⏸️ 워크플로우 일시정지!
        feedback = interrupt({
            "message": "검토 필요",
            "data": state["draft"]
        })
        # feedback은 resume 시 전달받음
        return {"review": feedback}
    
    # 검토 불필요시 자동 통과
    return {}
```

**재개 방법:**
```python
# 1차 실행: interrupt까지
result = app.invoke(input, config=config)
print(result["__interrupt__"])  # 인터럽트 페이로드

# 2차 실행: 피드백과 함께 재개
result = app.invoke(
    Command(resume="승인합니다"),
    config=config  # ⚠️ 같은 thread_id 필수!
)
```

### 🎯 실무 활용:
- **결재 워크플로우**: 상사 승인 필요
- **컨텐츠 검수**: AI 초안 → 인간 검토
- **민감 정보**: 개인정보 처리 전 확인

---

아래 셀에서는 간단한 검토 워크플로우를 만들어봅니다.


In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Step 4 데모: 간단한 HITL 워크플로우
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
from typing import TypedDict

class ReviewState(TypedDict):
    query: str
    draft: str
    review_feedback: str
    final: str
    needs_review: bool


def draft_writer(state: ReviewState) -> Dict:
    """초안 작성"""
    draft = f"고객님의 문의 '{state['query']}' 에 대한 답변입니다."
    print(f"📝 초안 작성: {draft}")
    return {"draft": draft}


def review_gate(state: ReviewState) -> Dict:
    """
    검토 게이트: 환불 키워드 → 인간 검토
    
    ⭐ interrupt() 사용:
    - 워크플로우 일시정지
    - 피드백 입력 대기
    - Command(resume=...) 로 재개
    """
    if state["needs_review"]:
        print("⏸️  인터럽트 발생! 검토자 피드백 대기...")
        
        # ⏸️ 워크플로우 일시정지!
        feedback = interrupt({
            "title": "검토 필요",
            "draft": state["draft"],
            "instruction": "승인 또는 수정 의견을 입력하세요"
        })
        
        print(f"▶️  재개됨! 피드백 수신: {feedback}")
        return {"review_feedback": feedback}
    
    # 검토 불필요 → 자동 통과
    print("✅ 자동 승인 (검토 불필요)")
    return {"review_feedback": "자동 승인"}


def finalizer(state: ReviewState) -> Dict:
    """최종 응답 생성"""
    final = state["draft"] + f"\n[검토: {state['review_feedback']}]"
    print(f"🎯 최종 응답: {final}")
    return {"final": final}


# HITL 그래프 구성
print("🧪 HITL 데모: interrupt로 워크플로우 제어\n")

hitl_graph = StateGraph(ReviewState)
hitl_graph.add_node("draft_writer", draft_writer)
hitl_graph.add_node("review_gate", review_gate)
hitl_graph.add_node("finalizer", finalizer)

hitl_graph.add_edge(START, "draft_writer")
hitl_graph.add_edge("draft_writer", "review_gate")
hitl_graph.add_edge("review_gate", "finalizer")
hitl_graph.add_edge("finalizer", END)

hitl_app = hitl_graph.compile(checkpointer=MemorySaver())

print("📊 HITL 그래프:")
visualize_graph(hitl_app)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 테스트 1: 자동 승인 경로
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print("\n" + "="*50)
print("테스트 1: 자동 승인 경로")
print("="*50)

auto_config = {"configurable": {"thread_id": "auto-1"}}
auto_input = ReviewState(
    query="제품 사용법 알려주세요",
    draft="",
    review_feedback="",
    final="",
    needs_review=False  # ← 검토 불필요
)

auto_result = hitl_app.invoke(auto_input, config=auto_config)
print(f"\n최종 결과: {auto_result['final']}")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 테스트 2: 인간 검토 경로 (interrupt)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print("\n" + "="*50)
print("테스트 2: 인간 검토 경로 (interrupt)")
print("="*50)

review_config = {"configurable": {"thread_id": "review-1"}}
review_input = ReviewState(
    query="환불 처리 부탁드립니다",
    draft="",
    review_feedback="",
    final="",
    needs_review=True  # ← 검토 필요!
)

# 1차 실행: interrupt까지
print("\n[1차 실행] interrupt까지 진행...")
first_run = hitl_app.invoke(review_input, config=review_config)
print(f"\n인터럽트 페이로드: {first_run.get('__interrupt__')}")

# 2차 실행: 피드백과 함께 재개
print("\n[2차 실행] 피드백 전달 후 재개...")
second_run = hitl_app.invoke(
    Command(resume="환불 승인합니다. 3-5 영업일 소요됩니다."),
    config=review_config  # ⚠️ 같은 thread_id!
)
print(f"\n최종 결과: {second_run['final']}")

print("\n" + "="*50)
print("💡 핵심:")
print("   - interrupt()로 워크플로우 일시정지")
print("   - Command(resume=...)로 재개")
print("   - 같은 thread_id로 상태 유지!")
print("="*50)


In [ ]:
def fan_in_node(state: IntegratedState) -> Dict:
    """
    Fan-in 노드: 3개 팀 보고서를 통합해서 초안 작성
    
    ⭐ 이 시점에서는 이미 State 병합 완료!
    - technical_report, policy_report, service_report 모두 존재
    - audit_log도 3개 팀의 로그가 모두 누적됨
    """
    reports = []
    if state['technical_report']:
        reports.append(state['technical_report'])
    if state['policy_report']:
        reports.append(state['policy_report'])
    if state['service_report']:
        reports.append(state['service_report'])
    
    draft = '\n\n'.join(reports)
    
    return {
        'draft_response': draft,
        'audit_log': ['Fan-in: 보고서 통합 완료'],
    }


print("✅ fan_in_node 정의 완료!")
print("   - 3개 팀 보고서 → 초안 생성")


In [ ]:
def quality_gate_node(state: IntegratedState) -> Command:
    """
    품질 가드: 정책 이슈는 인간 검토, 나머지는 자동 승인
    
    ⭐ interrupt 사용:
    - 정책 문의: interrupt()로 일시정지 → 피드백 대기
    - 기술/서비스: 자동 승인
    """
    if state['category'] == 'policy':
        # ⭐ 워크플로우 일시정지!
        feedback = interrupt({
            'title': '정책 검토 필요',
            'draft': state['draft_response'],
            'instruction': '검토 의견을 입력하면 워크플로우가 재개됩니다.',
        })
        
        return Command(
            goto='human_review',
            update={
                'review_feedback': feedback,  # interrupt에서 받은 피드백
                'audit_log': ['인간 검토 요청'],
            },
        )
    
    # 자동 승인 경로
    return Command(
        goto='writer_signoff',
        update={
            'final_response': state['draft_response'],
            'approval_status': 'auto',
            'audit_log': ['품질 가드 통과 (자동 승인)'],
        },
    )


def human_review_node(state: IntegratedState) -> Dict:
    """인간 검토: 피드백을 최종 응답에 반영"""
    final = '\n\n'.join([
        state['draft_response'],
        '[검토자 메모]',
        state['review_feedback']
    ])
    
    return {
        'final_response': final,
        'approval_status': 'approved',
        'audit_log': ['검토 피드백 반영'],
    }


def writer_signoff_node(state: IntegratedState) -> Dict:
    """최종 사인오프"""
    return {
        'audit_log': ['최종 사인오프 완료'],
    }


print("✅ HITL 노드 정의 완료!")
print("   - quality_gate: interrupt로 일시정지")
print("   - human_review: 피드백 반영")
print("   - writer_signoff: 최종 승인")

## Step 5. 전체 통합 (Full Graph)

### 🎯 모든 고급 패턴을 하나의 워크플로우로!

지금까지 배운 4가지 패턴을 통합합니다:

1. **Annotated State**: `audit_log` 자동 누적
2. **Command**: router가 동적으로 팀 선택
3. **Fan-in**: 3개 팀 보고서 → 하나로 합치기
4. **interrupt**: 정책 이슈는 인간 검토

#### 전체 워크플로우:
```
START
  ↓
router (Command로 팀 선택)
  ├→ technical_team
  ├→ policy_team
  └→ service_team
       ↓ (Fan-in)
    fan_in
       ↓
  quality_gate
       ↓ (interrupt if policy)
    ├→ human_review → writer_signoff
    └→ writer_signoff (자동 승인)
          ↓
        END
```

---

아래 셀에서는 전체 그래프를 조립하고 2가지 경로를 테스트합니다:
- **자동 승인 경로**: 기술 문의 → 자동 처리
- **인간 검토 경로**: 정책 문의 → interrupt → 재개

In [ ]:
full_graph = StateGraph(IntegratedState)

# 노드 등록
full_graph.add_node('router', router_node)
full_graph.add_node('technical_team', technical_team_node)
full_graph.add_node('policy_team', policy_team_node)
full_graph.add_node('service_team', service_team_node)
full_graph.add_node('fan_in', fan_in_node)
full_graph.add_node('quality_gate', quality_gate_node)
full_graph.add_node('human_review', human_review_node)
full_graph.add_node('writer_signoff', writer_signoff_node)

# 엣지 연결
full_graph.add_edge(START, 'router')

# ⚠️ 중요: router는 Command로 동적 라우팅하므로
# router → 팀들로 가는 고정 엣지를 만들면 안 됨!
# Command.goto가 알아서 라우팅함

# 각 팀 → fan_in (Fan-in 패턴)
full_graph.add_edge('technical_team', 'fan_in')
full_graph.add_edge('policy_team', 'fan_in')
full_graph.add_edge('service_team', 'fan_in')

# fan_in → quality_gate → writer_signoff
full_graph.add_edge('fan_in', 'quality_gate')

# quality_gate는 Command로 분기 (human_review or writer_signoff)
# 고정 엣지 없음!

# human_review → writer_signoff
full_graph.add_edge('human_review', 'writer_signoff')

# writer_signoff → END
full_graph.add_edge('writer_signoff', END)

full_app = full_graph.compile(checkpointer=MemorySaver())
print('✅ Full graph compiled!')
print('\n💡 핵심:')
print('   - router는 Command.goto로 1개 팀만 선택')
print('   - quality_gate도 Command.goto로 분기')
print('   - 고정 엣지가 없어야 Command가 작동!')

visualize_graph(full_app)

In [ ]:
print('[전체 그래프 - 자동 승인 경로]')
auto_config = {'configurable': {'thread_id': 'auto-demo'}}
auto_result = full_app.invoke(initial_state('충전기가 고장나서 도움이 필요해요'), config=auto_config)
print(auto_result['final_response'])
print('\n실행 로그:')
for step in auto_result['audit_log']:
    print('-', step)

### 실행 결과 1: 자동 승인 경로

기술 문의는 자동으로 처리됩니다:
- Router → technical_team → fan_in → quality_gate → writer_signoff
- 인간 검토 없이 바로 최종 응답 생성
- audit_log에 전체 처리 과정 기록

In [ ]:
print('[전체 그래프 - 인간 검토 경로]')
policy_config = {'configurable': {'thread_id': 'policy-case'}}
policy_first = full_app.invoke(initial_state('어제 환불을 요청했는데 아직 처리되지 않았어요.'), config=policy_config)
print('인터럽트 페이로드:')
print(policy_first['__interrupt__'])

policy_second = full_app.invoke(Command(resume='환불 승인 완료. 처리 기간은 영업일 기준 7일입니다.'), config=policy_config)
print('\n검토 후 최종 응답:')
print(policy_second['final_response'])
print('\n실행 로그:')
for step in policy_second['audit_log']:
    print('-', step)

### 실행 결과 2: 인간 검토
- 정책 문의에서는 인터럽트 페이로드를 통해 초안과 안내 문구를 검토자에게 전달합니다.
- `Command(resume=…)`로 재개하면 검토자 메모가 포함된 최종 응답이 생성되고, 실행 로그에 `인간 검토 요청 → 검토 피드백 반영 → 최종 사인오프` 순서가 기록됩니다.